In [23]:
import pandas as pd

# Nunca truncar colunas de texto
pd.set_option('display.max_colwidth', None)

# Mostrar todas as colunas e linhas, se quiser
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_excel('base_dados_original.xlsx')

# conta quantas classes diferentes cada descrição tem
contagem = df.groupby('descricao')['classe'].nunique()

# seleciona aquelas com mais de 1 classe
descricoes_inconsistentes = contagem[contagem > 1].index.tolist()

# filtra o df original para ver os registros problemáticos
df_conflitos = df[df['descricao'].isin(descricoes_inconsistentes)]
df_conflitos.sort_values(by='descricao')

,fonte,descricao,classe,Tipo


In [25]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.5 MB/s eta 0:00:00m eta 0:00:010:00:01


In [30]:
from rapidfuzz import process, fuzz

df = pd.read_excel('base_dados_original.xlsx')

# pega as descrições únicas
descricoes_unicas = df['descricao'].unique()

# define um limiar de similaridade (por exemplo, 90 de 100)
THRESHOLD = 90

pares_problematicos = []

# para cada descrição, busca as 5 mais similares dentro do seu próprio conjunto
for desc in descricoes_unicas:
    matches = process.extract(
        desc,
        descricoes_unicas,
        scorer=fuzz.token_sort_ratio,
        limit=5
    )
    for match_desc, score, _ in matches:
        if desc != match_desc and score >= THRESHOLD:
            # verifica se, nessas duas descrições, existe divergência de classes
            classes_desc  = set(df.loc[df['descricao'] == desc,  'classe'])
            classes_match = set(df.loc[df['descricao'] == match_desc, 'classe'])
            if classes_desc != classes_match:
                pares_problematicos.append(
                    (desc, match_desc, score, classes_desc, classes_match)
                )

# dicionário para unificação de pares (chave: tupla ordenada de descrições)
unicos = {}

for desc1, desc2, score, cls1, cls2 in pares_problematicos:
    key = tuple(sorted([desc1, desc2]))
    if key not in unicos:
        unicos[key] = (desc1, desc2, score, cls1, cls2)

# agora imprime
for desc1, desc2, score, cls1, cls2 in unicos.values():
    print(f"“{desc1}” ↔ “{desc2}” [{score}% similar] → classes {cls1} vs {cls2}")


“Contratacao de empresa especializada na prestacao de forma continua de servicos de limpeza conservacao higienizacao auxiliar de escritorio copeiragem vigia operador de cftv recepcionista auxiliar de almoxarifado servente lider auxiliar de servicos gerais e mensageiro objetivando a execucao no ambito da camara municipal de sao goncalo ” ↔ “Contratacao de empresa especializada na prestacao de forma continua de servicos de limpeza conservacao higienizacao auxiliar de escritorio copeiragem vigia operador de cftv recepcionista auxiliar de almoxarifado servente lider auxiliar de servicos gerais e mensageiro objetivando a execucao no ambito da camara municipal de sao goncalo conforme condicoes quantidades e exigencias estabelecidas neste ” [91.28065395095368% similar] → classes {'Não'} vs {'Sim'}
“Contratacao de empresa especializada na prestacao de servico de outsourcing de impressao com fornecimento de software de gerenciamento e bilhetagem acessorios suprimentos insumos consumiveis origin